In [12]:
import os
import pandas as pd
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, models
import torchvision.utils as vutils
import torchvision.transforms.functional as TF
from PIL import Image
from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, HiResCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import preprocess_image

In [13]:
# === Define device ===
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
# === RESNET18 architecture recreation ===
def load_resnet18_model(model_path, num_classes=2, device=torch.device("cpu")):
    # Load base ResNet-18 model
    model = models.resnet18(weights=None)
    # Modify first convolutional layer for grayscale input
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    # Modify the final fully connected layer for binary classification
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2) 
    # Move the model to the correct device (CPU or GPU)
    model.to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    # Set the model to evaluation mode (important for inference)
    model.eval()
    return model

In [15]:
def load_densenet169_model(model_path, num_classes=2, device=torch.device("cpu")):
    # Load base DenseNet-169 model
    model = models.densenet169(weights=None)
    # Modify first convolutional layer for grayscale input
    model.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    # Modify the final classifier for binary classification
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 2) 
    # Move the model to the correct device (CPU or GPU)
    model.to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    # Set the model to evaluation mode (important for inference)
    model.eval()
    return model

In [16]:
# === Define Image Transformations === (Ensure grayscale input and correct format) ************
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((80,80)),  # Match model's input size
    transforms.ToTensor(),
])

In [17]:
# === Extract Ground Truth from folder path ===
def get_ground_truth(img_name, folder):
    # Combine the folder path and image name for a comprehensive check
    full_path = os.path.join(folder, img_name)
    
    # Check for the ground truth in a case-insensitive way
    if "cesarean" in full_path.lower():
        return "Cesarean Birth"
    elif "vaginal" in full_path.lower():
        return "Vaginal Birth"
    else:
        print(f"Warning: Could not determine ground truth for {full_path}. Assuming 'Cesarean Birth'.")
        ground_truth = "Cesarean Birth"

    # Print the ground truth for each image
    #print(f"[Ground Truth] Image: {img_name} | Path: {full_path} | Picked: {ground_truth}")
    return ground_truth

In [18]:
# === Prediction Frame ===
def add_colored_frame(image, color, thickness=20):
    height, width = image.shape[:2]
    cv2.rectangle(image, (0, 0), (width - 1, height - 1), color, thickness)
    return image

In [19]:
# === Heatmap Overlay ===
def overlay_heatmap(original_image, heatmap): #heatmap w\ size [1,3,3]
    heatmap = cv2.resize(heatmap[0], (original_image.shape[1], original_image.shape[0])) #heatmap size from [3,3] to [80,80]
    heatmap = np.uint8(255 * heatmap) #adapt to image format needed for color mapping
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) #apply color map

    if len(original_image.shape) == 2 or original_image.shape[2] == 1:
        original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)

    #original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR) # Convert grayscale to 3-channel image
    overlay = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0) #blend - 60% of grayscale image , 40% for colored heatmap
    return overlay

In [ ]:
def apply_gradcam_and_save(model, input_folder, output_folder, grid_image_size=(80, 80)):
    os.makedirs(output_folder, exist_ok=True)

    target_layers = [model.features.denseblock4.denselayer32]  #layer4 for ResNet-18

    original_images = []
    correct_heatmaps = []
    incorrect_heatmaps = []
    misclassified_tensors = []
    correctly_classified_tensors = []

    class_names = ["Cesarean Birth", "Vaginal Birth"]

    for img_name in os.listdir(input_folder):
        if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(input_folder, img_name)
        image = Image.open(img_path).convert('L')
        input_tensor = transform(image).unsqueeze(0).to(device)
        resized_image = transforms.Resize((80, 80))(image)
        original_np = np.array(resized_image)
        original_images.append(original_np)

        # Previsão para obter classe
        with torch.no_grad():
            output = model(input_tensor)
            prediction = output.argmax(dim=1).item()

        # Grad-CAM
        with GradCAM(model=model, target_layers=target_layers) as cam:   #use_cuda=torch.cuda.is_available()
            targets = [ClassifierOutputTarget(prediction)]
            grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]  # [H, W]

        # === Ground truth ===
        ground_truth_label = get_ground_truth(img_name, input_folder)
        ground_truth_class = class_names.index(ground_truth_label)

        # === Escolher cor frame ===
        frame_color = (0, 255, 0) #green
        if prediction != ground_truth_class:
            frame_color = (0, 0, 255)   #red
    

        # === Overlay do Grad-CAM ===
        overlay = overlay_heatmap(original_np, grayscale_cam[np.newaxis, ...])  # adiciona dimensão extra
        overlay_with_frame = add_colored_frame(overlay, frame_color, thickness=7)

        save_path = os.path.join(output_folder, f"heatmap_{class_names[prediction]}_{img_name}")
        cv2.imwrite(save_path, overlay_with_frame)
        print(f"Saved: {save_path} ; Predicted: {class_names[prediction]}")

        # === Grid de resultados ===
        overlay_rgb = cv2.cvtColor(overlay_with_frame, cv2.COLOR_BGR2RGB)
        overlay_pil = Image.fromarray(overlay_rgb).resize(grid_image_size)
        tensor_for_grid = TF.to_tensor(overlay_pil)

        if prediction == ground_truth_class:
            correctly_classified_tensors.append(tensor_for_grid)
            correct_heatmaps.append(grayscale_cam)  # já está com shape [H, W]
        else:
            misclassified_tensors.append(tensor_for_grid)
            incorrect_heatmaps.append(grayscale_cam)

    # === Geração do Grid ===
    final_sorted_grid = misclassified_tensors + correctly_classified_tensors
    if final_sorted_grid:
        grid = vutils.make_grid(final_sorted_grid, nrow=20, padding=5, normalize=True)
        grid_path = os.path.join(output_folder, "grid_overlay_ordered.jpg")
        vutils.save_image(grid, grid_path)
        print(f"Grid saved at: {grid_path}")

    # === Heatmaps Médios (acertos) ===
    if correct_heatmaps:
        mean_heatmap = np.mean(np.stack(correct_heatmaps), axis=0)
        mean_heatmap -= mean_heatmap.min()
        mean_heatmap /= (mean_heatmap.max() + 1e-8)
        heatmap_uint8 = np.uint8(255 * mean_heatmap)
        heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)

        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_NEAREST)
        heatmap_resized_smooth = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_CUBIC)

        mean_path_blocky = os.path.join(output_folder, "correct_mean_heatmap_blocky.png")
        cv2.imwrite(mean_path_blocky, heatmap_resized_blocky)
        print(f"Mean heatmap (correct only) saved at: {mean_path_blocky}")

        mean_path_smooth = os.path.join(output_folder, "correct_mean_smooth.png")
        cv2.imwrite(mean_path_smooth, heatmap_resized_smooth)
        print(f"Mean heatmap (correct only) saved at: {mean_path_smooth}")

    # === Heatmaps Médios (erros) ===
    if incorrect_heatmaps:
        mean_heatmap_incorrect = np.mean(np.stack(incorrect_heatmaps), axis=0)
        mean_heatmap_incorrect -= mean_heatmap_incorrect.min()
        mean_heatmap_incorrect /= (mean_heatmap_incorrect.max() + 1e-8)
        heatmap_uint8_incorrect = np.uint8(255 * mean_heatmap_incorrect)
        heatmap_colored_incorrect = cv2.applyColorMap(heatmap_uint8_incorrect, cv2.COLORMAP_JET)

        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_NEAREST)
        heatmap_resized_smooth_incorrect = cv2.resize(heatmap_colored_incorrect, (w, h), interpolation=cv2.INTER_CUBIC)

        mean_path_blocky_incorrect = os.path.join(output_folder, "incorrect_mean_heatmap_blocky.png")
        cv2.imwrite(mean_path_blocky_incorrect, heatmap_resized_blocky_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_blocky_incorrect}")

        mean_path_smooth_incorrect = os.path.join(output_folder, "incorrect_mean_smooth.png")
        cv2.imwrite(mean_path_smooth_incorrect, heatmap_resized_smooth_incorrect)
        print(f"Mean heatmap (incorrect only) saved at: {mean_path_smooth_incorrect}")

    return correct_heatmaps, original_images


In [21]:
def generate_prediction_table_only(model, input_folder, split_name, structure, birth_type, dataset):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.Resize((80,80)),  # Match model's input size
        transforms.ToTensor(),
    ])

    class_names = ["Cesarean Birth", "Vaginal Birth"]
    prediction_results = []

    for img_name in os.listdir(input_folder):
        if not img_name.lower().endswith(".png"):
            continue

        img_path = os.path.join(input_folder, img_name)
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            prediction = output.argmax(dim=1).item()
            probabilities = torch.softmax(output, dim=1)
            confidence = probabilities[0, prediction].item()

        ground_truth_label = get_ground_truth(img_name, input_folder)
        ground_truth_class = class_names.index(ground_truth_label)

        prediction_results.append({
            "Image": img_name,
            "Predicted Class": class_names[prediction],
            "Ground Truth": ground_truth_label,
            "Correct": prediction == ground_truth_class,
            "Confidence": round(confidence, 4),
            "Split": split_name,
            "Structure": structure,
            "Birth Type": birth_type,
            "Dataset": dataset
        })

    return pd.DataFrame(prediction_results)


In [23]:
def main():

    only_generate_csv = True

    structures = ["Abdomen_","Femur_", "Head_"] 
    birth_types = ["Cesarean Birth", "Vaginal Birth"]
    datasets = ["train", "test"]
    splits = ["cv1","cv2","cv3"] 

    base_input = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/image-dataset/dataset_images_{split}/{structure}/{dataset}/{birth_type}"
    base_output = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/HiRes_DENSENET169/HiRes_densenet_{split}_results/X_HiRes_densenet_{structure_clean}_{birth_type_short}_{dataset}_{split}"
    base_weight = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/model_paths/DenseNet_paths/{structure_clean}_{split}_densenet169_best-model.pth"
    output_dir = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/prediction_tables"
    os.makedirs(output_dir, exist_ok=True)

    excel_save_path = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/predictions_confidence_resnet18.xlsx"

    if only_generate_csv:
        dfs_by_split = {split: [] for split in splits}
        
        for split in splits:
            for structure in structures:
                structure_clean = structure.rstrip("_").lower()
                for birth_type in birth_types:
                    birth_type_short = birth_type.split()[0].lower()  # 'cesarean' or 'vaginal'
                    for dataset in datasets:
                        input_folder = base_input.format(split=split, structure=structure, birth_type=birth_type, dataset=dataset)
                        #output_folder = base_output.format(split=split, structure_clean=structure_clean, birth_type_short=birth_type_short, dataset=dataset)
                        weight_path = base_weight.format(structure_clean=structure_clean, split=split)

                        if not os.path.exists(weight_path):
                            print(f"[SKIPPED] Weight file does not exist: {weight_path}")
                            continue

                        print(f"Processing: {structure}, {birth_type}, {dataset}, {split}")
                        model = load_resnet18_model(weight_path, num_classes=2)
                        #all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)

                        df = generate_prediction_table_only(
                            model, 
                            input_folder, 
                            split_name=split,
                            structure=structure_clean,
                            birth_type=birth_type_short,
                            dataset=dataset
                        )
                        dfs_by_split[split].append(df)
                    
        # === GUARDAR CSVs ===
        for split, df_list in dfs_by_split.items():
            if df_list:
                combined_df = pd.concat(df_list, ignore_index=True)
                csv_path = os.path.join(output_dir, f"{split}.csv")
                combined_df.to_csv(csv_path, index=False)
                print(f"Guardado: {csv_path}")

    else:
        print("Pipeline completo (com heatmaps) não ativado. Altere `only_generate_csv` para False se necessário.")

    print("All Grad-CAM heatmaps were generated successfully.")

if __name__ == "__main__":
    main()


Processing: Abdomen_, Cesarean Birth, train, cv1


C:\Users\anale\AppData\Local\Temp\ipykernel_7820\3343887526.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=de

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer2.0.conv1.weight", "layer2.0.bn1.weight", "layer2.0.bn1.bias", "layer2.0.bn1.running_mean", "layer2.0.bn1.running_var", "layer2.0.conv2.weight", "layer2.0.bn2.weight", "layer2.0.bn2.bias", "layer2.0.bn2.running_mean", "layer2.0.bn2.running_var", "layer2.0.downsample.0.weight", "layer2.0.downsample.1.weight", "layer2.0.d